In [2]:
import functools
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels as sm
import scipy.stats as sct
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_blobs, make_classification
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.naive_bayes import GaussianNB

df_train = pd.read_csv("C:/Users/caioe/codenation/enem-4/train.csv", sep = ",")
df_test = pd.read_csv("C:/Users/caioe/codenation/enem-4/test.csv", sep = ",")

C:\Users\caioe\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, defaultdict
C:\Users\caioe\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Columns: 167 entries, Unnamed: 0 to Q050
dtypes: float64(28), int64(79), object(60)
memory usage: 17.5+ MB


In [4]:
df_train.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,IN_OUTRA_DEF,IN_SABATISTA,IN_GESTANTE,IN_LACTANTE,IN_IDOSO,IN_ESTUDA_CLASSE_HOSPITALAR,IN_SEM_RECURSO,IN_BRAILLE,IN_AMPLIADA_24,IN_AMPLIADA_18,IN_LEDOR,IN_ACESSO,IN_TRANSCRICAO,IN_LIBRAS,IN_LEITURA_LABIAL,IN_MESA_CADEIRA_RODAS,IN_MESA_CADEIRA_SEPARADA,IN_APOIO_PERNA,IN_GUIA_INTERPRETE,IN_MACA,IN_COMPUTADOR,IN_CADEIRA_ESPECIAL,IN_CADEIRA_CANHOTO,IN_CADEIRA_ACOLCHOADA,IN_PROVA_DEITADO,IN_MOBILIARIO_OBESO,IN_LAMINA_OVERLAY,IN_PROTETOR_AURICULAR,IN_MEDIDOR_GLICOSE,IN_MAQUINA_BRAILE,IN_SOROBAN,IN_MARCA_PASSO,IN_SONDA,IN_MEDICAMENTOS,IN_SALA_INDIVIDUAL,IN_SALA_ESPECIAL,IN_SALA_ACOMPANHANTE,IN_MOBILIARIO_ESPECIFICO,IN_MATERIAL_ESPECIFICO,IN_NOME_SOCIAL,IN_CERTIFICADO,NO_ENTIDADE_CERTIFICACAO,CO_UF_ENTIDADE_CERTIFICACAO,SG_UF_ENTIDADE_CERTIFICACAO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,1,4314902.0,Porto Alegre,43.0,RS,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,4314902,Porto Alegre,43,RS,1,1,1,1,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,436.3,495.4,581.2,399.4,ABACCABDBEDEDEEDDDBBBDAABABECDCCCAAABCBEEABCA,BBEBCAEBCEDBDDEBCACDBCBDABDDCDBEDCEAEABCAADAC,99999DCABCABCBBCADBCBBABAEADBEEBEDAAABAEAECCBB...,AECCCBABDCADDBCBCCADADBBAADEBCEEBABCDDEAABCAA,1,BCADBCEDCCBADBDCADBEBDBEADCCAEACDBEDBCACAEBEE,BCBCCADCCBEDDBEEADBDCECCBBECAEEDDADCDDACAADBA,CEDEACCBEBDBABECABDADEADCEDCBEBBCEBAAEBAAECBBC...,DADCCECBDECADCBACDBCEEABDDAABDBCEADDBECEBAEBC,1.0,120.0,120.0,120.0,80.0,80.0,520.0,D,D,B,B,3,C,A,B,B,B,A,B,A,A,A,A,A,A,B,A,A,C,A,A,A,C,H,E,5.0,5.0,1.0,0.0,0.0,5,5,0,5,5,5,5,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,1,2304707.0,Granja,23.0,CE,2,0,2,1.0,0,23005157.0,2304707.0,Granja,23.0,CE,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2304707,Granja,23,CE,1,1,1,1,b9b06ce8c319a3df2158ea3d0aef0f7d3eecaed7,909237ab0d84688e10c0470e2997348aff585273,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,474.5,544.1,599.0,459.8,EDAAABDABADBCCAEDCDDECABADBEECBBAEEDCABCAABBC,ADCAABAADCEDAACDACEEDACBDCBACEDEDBACAADADABDC,99999ACBACDBDBCEADDAEEADCCEACEEDBECAAAACBECDAC...,AEDACBDECEDAEEAEDDEBDECDCECACADCACACAABDACEAB,1,EBEECADBCABECDDBEADCCABDEABEBCADBADDCCCEDBBCA,DCECBAAADCBBECAEDCEDDACCBDDACEDDBADBCADBCBCEE,CEDEACCBEBDBABECABDADEADCEDCBEBBCEBAAEBAAECBBC...,DADCCECBDECADCBACDBCEEABDDAABDBCEA

In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4570 entries, 0 to 4569
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NU_INSCRICAO            4570 non-null   object 
 1   CO_UF_RESIDENCIA        4570 non-null   int64  
 2   SG_UF_RESIDENCIA        4570 non-null   object 
 3   NU_IDADE                4570 non-null   int64  
 4   TP_SEXO                 4570 non-null   object 
 5   TP_COR_RACA             4570 non-null   int64  
 6   TP_NACIONALIDADE        4570 non-null   int64  
 7   TP_ST_CONCLUSAO         4570 non-null   int64  
 8   TP_ANO_CONCLUIU         4570 non-null   int64  
 9   TP_ESCOLA               4570 non-null   int64  
 10  TP_ENSINO               1426 non-null   float64
 11  TP_DEPENDENCIA_ADM_ESC  1426 non-null   float64
 12  IN_BAIXA_VISAO          4570 non-null   int64  
 13  IN_CEGUEIRA             4570 non-null   int64  
 14  IN_SURDEZ               4570 non-null   

In [10]:
round(df_test.isnull().sum()/4570,2)

NU_INSCRICAO              0.00
CO_UF_RESIDENCIA          0.00
SG_UF_RESIDENCIA          0.00
NU_IDADE                  0.00
TP_SEXO                   0.00
TP_COR_RACA               0.00
TP_NACIONALIDADE          0.00
TP_ST_CONCLUSAO           0.00
TP_ANO_CONCLUIU           0.00
TP_ESCOLA                 0.00
TP_ENSINO                 0.69
TP_DEPENDENCIA_ADM_ESC    0.69
IN_BAIXA_VISAO            0.00
IN_CEGUEIRA               0.00
IN_SURDEZ                 0.00
IN_DISLEXIA               0.00
IN_DISCALCULIA            0.00
IN_SABATISTA              0.00
IN_GESTANTE               0.00
IN_IDOSO                  0.00
TP_PRESENCA_CN            0.00
TP_PRESENCA_CH            0.00
TP_PRESENCA_LC            0.00
TP_PRESENCA_MT            0.00
NU_NOTA_CN                0.24
NU_NOTA_CH                0.24
NU_NOTA_LC                0.26
TP_LINGUA                 0.00
TP_STATUS_REDACAO         0.26
NU_NOTA_COMP1             0.26
NU_NOTA_COMP2             0.26
NU_NOTA_COMP3             0.26
NU_NOTA_

In [18]:
features = ['NU_INSCRICAO',
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'NU_IDADE',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'TP_DEPENDENCIA_ADM_ESC',
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DISLEXIA',
 'IN_DISCALCULIA',
 'IN_SABATISTA',
 'IN_GESTANTE',
 'IN_IDOSO',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q006',
 'Q024',
 'Q025',
 'Q026',
 'Q027',
 'Q047']

In [21]:
df_train = df_train[features]

In [23]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NU_INSCRICAO            13730 non-null  object 
 1   CO_UF_RESIDENCIA        13730 non-null  int64  
 2   SG_UF_RESIDENCIA        13730 non-null  object 
 3   NU_IDADE                13730 non-null  int64  
 4   TP_SEXO                 13730 non-null  object 
 5   TP_COR_RACA             13730 non-null  int64  
 6   TP_NACIONALIDADE        13730 non-null  int64  
 7   TP_ST_CONCLUSAO         13730 non-null  int64  
 8   TP_ANO_CONCLUIU         13730 non-null  int64  
 9   TP_ESCOLA               13730 non-null  int64  
 10  TP_ENSINO               4282 non-null   float64
 11  IN_TREINEIRO            13730 non-null  int64  
 12  TP_DEPENDENCIA_ADM_ESC  4282 non-null   float64
 13  IN_BAIXA_VISAO          13730 non-null  int64  
 14  IN_CEGUEIRA             13730 non-null

In [23]:
features_1 = ['NU_IDADE',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_LC',
 'Q047']

In [24]:
df_train_small = df_train[features_1]

In [25]:
df_train_small.head()

,NU_IDADE,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_PRESENCA_CN,TP_PRESENCA_LC,Q047
0,24,1,1,4,1,NaN,0,1,1,A
1,17,1,2,0,2,1.0,0,1,1,A
2,21,1,3,0,1,NaN,0,0,0,A
3,25,1,1,9,1,NaN,0,0,0,D
4,28,1,1,4,1,NaN,0,0,0,A


In [35]:
df_train_small.groupby('NU_IDADE')['IN_TREINEIRO'].value_counts().unstack().fillna(0)

IN_TREINEIRO,0,1
NU_IDADE,,
13,0.0,1.0
14,7.0,14.0
15,4.0,268.0
16,157.0,861.0
17,2032.0,598.0
18,2237.0,41.0
19,1455.0,0.0
20,1036.0,0.0
21,757.0,0.0


In [36]:
df_train_small.groupby('TP_NACIONALIDADE')['IN_TREINEIRO'].value_counts().unstack().fillna(0)

IN_TREINEIRO,0,1
TP_NACIONALIDADE,,
0,11,2
1,11406,1715
2,485,63
3,7,1
4,38,2


In [37]:
df_train_small.groupby('TP_ST_CONCLUSAO')['IN_TREINEIRO'].value_counts().unstack().fillna(0)

IN_TREINEIRO,0,1
TP_ST_CONCLUSAO,,
1,6524.0,0.0
2,4283.0,0.0
3,523.0,1783.0
4,617.0,0.0


In [39]:
df_train_small.groupby('TP_ANO_CONCLUIU')['IN_TREINEIRO'].value_counts().unstack().fillna(0)

IN_TREINEIRO,0,1
TP_ANO_CONCLUIU,,
0,5423.0,1783.0
1,1578.0,0.0
2,1033.0,0.0
3,712.0,0.0
4,563.0,0.0
5,379.0,0.0
6,383.0,0.0
7,294.0,0.0
8,221.0,0.0


In [6]:
df_train_small.groupby('TP_PRESENCA_CN')['IN_TREINEIRO'].value_counts().unstack().fillna(0)

IN_TREINEIRO,0,1
TP_PRESENCA_CN,,
0,3130,250
1,8809,1532
2,8,1


In [7]:
df_train_small.groupby('TP_PRESENCA_LC')['IN_TREINEIRO'].value_counts().unstack().fillna(0)

IN_TREINEIRO,0,1
TP_PRESENCA_LC,,
0,3314.0,278.0
1,8628.0,1505.0
2,5.0,0.0


In [41]:
df_train_small.groupby('Q047')['IN_TREINEIRO'].value_counts().unstack().fillna(0)

IN_TREINEIRO,0,1
Q047,,
A,10293,1433
B,318,30
C,152,10
D,924,249
E,260,61


In [26]:
df_train_small.groupby('TP_ESCOLA')['IN_TREINEIRO'].value_counts().unstack().fillna(0)

IN_TREINEIRO,0,1
TP_ESCOLA,,
1,7664.0,1783.0
2,3713.0,0.0
3,569.0,0.0
4,1.0,0.0


In [42]:
len(df_train_small[df_train_small['IN_TREINEIRO'] == 1])

1783

In [7]:
features_class = ['NU_IDADE',
 'TP_ST_CONCLUSAO',
 'IN_TREINEIRO']

In [8]:
df_train_optimal = df_train[features_class]
df_train_optimal = pd.get_dummies(df_train_optimal, columns = ['TP_ST_CONCLUSAO'])

In [39]:
list(df_train_optimal)

['IN_TREINEIRO',
 'TP_ST_CONCLUSAO_1',
 'TP_ST_CONCLUSAO_2',
 'TP_ST_CONCLUSAO_3',
 'TP_ST_CONCLUSAO_4']

In [30]:
features_class_1 = ['NU_IDADE',
 'TP_ST_CONCLUSAO_3']

In [36]:
y = df_train_optimal[['IN_TREINEIRO']]
X = df_train_optimal[features_class_1]
#X_test = df_train_optimal[features_class_1]

In [ ]:
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.fit_transform(X_test)

In [37]:
smote = SMOTE()

X_balanced, y_balanced = smote.fit_sample(X, y)

C:\Users\caioe\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
(len(X_balanced[X_balanced == 0]), len(X_balanced[X_balanced == 1]))

(11424, 12470)

In [19]:
(len(y_balanced[y_balanced == 0]), len(y_balanced[y_balanced == 1]))

(11947, 11947)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2)

In [59]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
lr = f1_score(y_test, y_pred)
print(cm)
print(f1_score(y_test, y_pred))

[[2296   75]
 [   0 2408]]
0.9846657125332243


In [57]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
rf = f1_score(y_test, y_pred)
print(cm)
print(f1_score(y_test, y_pred))

[[2316   55]
 [   0 2408]]
0.98870868404845


In [51]:
importances = rf_model.feature_importances_
importances

array([0.4407403, 0.5592597])

In [55]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
knn = f1_score(y_test, y_pred)
print(cm)
print(f1_score(y_test, y_pred))

[[2316   55]
 [   0 2408]]
0.98870868404845


In [56]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

y_pred = nb_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
nb = f1_score(y_test, y_pred)
print(cm)
print(f1_score(y_test, y_pred))

[[2274   97]
 [   0 2408]]
0.9802564624465703


In [61]:
models = pd.DataFrame({'models' : ['Logistic Regression', 'Random Forest Classifier', 'K Neighbors Classifier', 'Gaussian'],
                   'score' : [lr, rf, knn, nb]})
models

,models,score
0,Logistic Regression,0.984666
1,Random Forest Classifier,0.988709
2,K Neighbors Classifier,0.988709
3,Gaussian,0.980256


In [44]:
df_test_optimal = df_test[['NU_INSCRICAO','NU_IDADE','TP_ST_CONCLUSAO']]
df_test_optimal = pd.get_dummies(df_test_optimal, columns = ['TP_ST_CONCLUSAO'])

In [53]:
df_test_optimal = df_test_optimal[['NU_INSCRICAO','NU_IDADE','TP_ST_CONCLUSAO_3']]
Y_test = df_test_optimal[['NU_IDADE','TP_ST_CONCLUSAO_3']]

y_pred = rf_model.predict(Y_test)

df_test_optimal['IN_TREINEIRO'] = y_pred

answer = df_test_optimal[['NU_INSCRICAO','IN_TREINEIRO']]

answer

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0
...,...,...
4565,361b7fcd8867119550fe2af5aa729ffad89a7cf5,0
4566,d8a0e4c9e29494cc9bba2422bd79333931475ee1,0
4567,3f1c3388244df8d6521e983a809292d9f3bca643,0
4568,1778e9c4cef591beb6b986d191d15ed05de816b0,0


In [54]:
write = answer.to_csv("C:/Users/caioe/codenation/enem-4/answer.csv", index = False)